# How do I get billing information for a period?
### Overview
Billing information on the Seven Bridges platforms is broken down to three categories:
- Analysis
- Storage
- Egress

For details on how all the costs are calculated, please refer to the Seven Bridges documentation.

This recipe will show how to:
- Select a billing group
- Get analysis costs breakdown for a time period and total analysis costs
- Get storage costs breakdown for a time period and total storage costs
- Get egress costs breakdown for a time period and total egress costs

### Prerequisites
 1. You need to be an admin of _at least one_ billing group.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 
## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The `Api` object needs to know your **auth\_token** and the correct path. Here we assume you are using the credentials file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify credentials file profile {cgc, sbg, default}
prof = 'default'

config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Select a billing group

Here we will just take the first billing group from the list of available groups. If you are working in a division, there will always be a single billing group.

In [ ]:
bg = api.billing_groups.query()[0]

## Analysis breakdown

Breakdown of the analysis costs in a specific period is available by invoking `analysis_breakdown()` method with fields `date_from` and `date_to` set in format `dd-mm-yyyy`. The result will be a list of all analyses that incurred cost, with details on who ran an analysis, when, in which project and how much did it cost.

Here we get a single analysis item and show some of the details (Note that you need to have at least one analysis in the specified period for the code to work)

In [ ]:
try:
    analysis = bg.analysis_breakdown(date_from='01-01-2020', date_to='01-09-2021')[0]
    print('App {} executed by {} in project {} cost {}'.format(
        analysis.analysis_app_name,
        analysis.ran_by,
        analysis.project_name,
        analysis.analysis_cost.amount
    ))
except IndexError:
    print('No analysis costs found')

## Storage breakdown

Breakdown of the analysis costs in a specific period is available by invoking `storage_breakdown()` method with fields `date_from` and `date_to` set in format `dd-mm-yyyy`. The result will be a list of all projects that incurred storage cost, with details such as total GB * month of storage used for active and archived files, project name, project creator and the project location. Each file contributes to the total GB * months by multiplying file size and file lifetime. For example, if a month has 30 days, file has 15 GB and it was in the project for 20 days before it was deleted, it will contribute with 10 GB*months.

Based on the project location and the cloud provider, price per unit may vary. The exact calculation of storage costs is not trivial, and different cloud providers offer different discounts with increased storage usage. 
For the purpose of this recipe, we are applying a simple but fairly precise calculation by using a single storage price for active and a single storage price for archived files. The storage prices can be obtained from cloud provider website, e.g. (https://aws.amazon.com/s3/pricing/).

Here we get a single project and show some of the details. (Note: you need at least some storage costs for this to work)

In [ ]:
active_storage_price = 0.023 # $/(GB * month)
archived_storage_price = 0.004 # $/(GB * month)
try:
    storage = bg.storage_breakdown(date_from='01-01-2020', date_to='01-09-2021')[0]
    print('In project {}, created by {}, hosted on location {}, total of {} of active GB*month and {} of archived GB*month '.format(
        storage.project_name,
        storage.project_created_by,
        storage.location,
        storage.active.size,
        storage.archived.size
    ))
    project_storage_cost = storage.active.size * active_storage_price + storage.archived.size * archived_storage_price
    print('Total storage cost for project {} was {}$'.format(storage.project_name, project_storage_cost))
except IndexError:
    print('No storage costs found')

## Egress breakdown

Breakdown of the egress costs in a specific period is available by invoking `egress_breakdown()` method with fields `date_from` and `date_to` set in format `dd-mm-yyyy`. The result will be a list of all file downloads that incurred cost, with details such as file size, downloader and download cost.

Here we get a single download and show some of the details. (Note, for this to work you need at least one egress cost)

In [ ]:
try:
    egress = bg.egress_breakdown(date_from='01-01-2020', date_to='01-09-2021')[0]
    print('User {} downloaded a file with size {} and it cost {}'.format(egress.downloaded_by, egress.downloaded.size, egress.egress_cost.amount))
except IndexError:
    print('No egress costs found')

## Total costs in a specific period

To find all costs incurred in a period, iterate through all breakdown items and sum it all up

In [ ]:
date_from = '01-01-2020'
date_to = '01-09-2021'
 
# Analysis cost total
analysis_total = sum([float(a.analysis_cost.amount) for a in bg.analysis_breakdown(date_from=date_from, date_to=date_to).all()])

# Storage cost total
active_storage_price = 0.023 # $/(GB * month)
archived_storage_price = 0.004 # $/(GB * month)
storage_total = sum([storage.active.size * active_storage_price + storage.archived.size * archived_storage_price
                 for storage in bg.storage_breakdown(date_from=date_from, date_to=date_to)])

# Egress cost total
egress_total = sum([float(e.egress_cost.amount) for e in bg.egress_breakdown(date_from=date_from, date_to=date_to).all()])

total = analysis_total + storage_total + egress_total

print('Analysis total : {}$'.format(analysis_total))
print('Storage total : {}$'.format(storage_total))
print('Egress total : {}$'.format(egress_total))
print('Total : {}$'.format(total))